In [17]:
from tensorflow.keras.models import load_model

try:
    model = load_model("LettuceModel_SavedModel")
    print("Model loaded successfully!")
except Exception as e:
    print("Error loading model:", e)


Model loaded successfully!


In [18]:
import tensorflow as tf

def check_model_type(model_path):
    try:
        # Try to load as a full model
        model = tf.keras.models.load_model(model_path)
        print("✅ The .h5 file is a FULL MODEL (architecture + weights).")
    except Exception as e:
        print("❌ Failed to load as a full model. Checking if it's weights only...")

        try:
            # Try loading as weights only
            model = tf.keras.models.Sequential()  # Create a dummy model
            model.load_weights(model_path)
            print("✅ The .h5 file contains ONLY WEIGHTS.")
        except Exception as e:
            print("❌ The file is neither a full model nor valid weights.")
            print("Error:", str(e))

# 🔍 Run the check (Change 'your_model.h5' to your actual filename)
check_model_type("LettuceModel_SavedModel")


✅ The .h5 file is a FULL MODEL (architecture + weights).


In [19]:
import h5py

file_path = "V3Model.h5"

with h5py.File(file_path, "r") as f:
    if "model_weights" in f.keys():
        print("✅ The file contains model weights.")
    if "keras_version" in f.attrs:
        print("✅ The file contains a full model (architecture + weights).")


✅ The file contains model weights.
✅ The file contains a full model (architecture + weights).


In [16]:
# Save as SavedModel format
model.save("LettuceModel_SavedModel")

# Load back
loaded_model = tf.keras.models.load_model("LettuceModel_SavedModel")
print("✅ Successfully loaded from SavedModel format!")


INFO:tensorflow:Assets written to: LettuceModel_SavedModel\assets


INFO:tensorflow:Assets written to: LettuceModel_SavedModel\assets


✅ Successfully loaded from SavedModel format!


In [14]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, Dropout, BatchNormalization, Add, Input

# ========================
# 🏗️ Rebuild the Model (MUST match original structure!)
# ========================
def build_model():
    input_layer = Input(shape=(224, 224, 3))

    # Block 1
    x = Conv2D(32, (3, 3), padding='same', activation='relu', kernel_initializer='he_normal')(input_layer)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)

    # Block 2
    x = Conv2D(64, (3, 3), padding='same', activation='relu', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)

    # Block 3 (Adding Residual Connection)
    x = Conv2D(128, (3, 3), padding='same', activation='relu', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)
    shortcut = x
    x = Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Add()([x, shortcut])

    # Block 4
    x = Conv2D(256, (3, 3), padding='same', activation='relu', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)

    # Extra Convolutional Block (For Better Feature Extraction)
    x = Conv2D(512, (3, 3), padding='same', activation='relu', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)

    # Global Pooling instead of Flatten
    x = GlobalAveragePooling2D()(x)

    # Fully Connected Layers
    x = Dense(256, activation='relu', kernel_initializer='he_normal')(x)
    x = Dropout(0.3)(x)

    x = Dense(128, activation='relu', kernel_initializer='he_normal')(x)
    x = Dropout(0.3)(x)

    # ✅ Corrected Output Layer (Connected to `x`)
    output_layer = Dense(4, activation='softmax')(x)  # 4 Classes: Healthy, Fungal, Bacterial, Lettuce

    model = Model(inputs=input_layer, outputs=output_layer)
    
    return model

# 🔄 Rebuild the model
model = build_model()
model.summary()  # Check if it matches the original

# 🏋️ Load the saved weights (replace with your file name)
model.load_weights("LettuceModel.h5")

print("✅ Model rebuilt & weights loaded successfully!")  


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d_6 (Conv2D)           (None, 224, 224, 32)         896       ['input_3[0][0]']             
                                                                                                  
 batch_normalization_6 (Bat  (None, 224, 224, 32)         128       ['conv2d_6[0][0]']            
 chNormalization)                                                                                 
                                                                                                  
 max_pooling2d_5 (MaxPoolin  (None, 112, 112, 32)         0         ['batch_normalization_6[

ValueError: Layer count mismatch when loading weights from file. Model expected 15 layers, found 4 saved layers.